In [3]:
from sklearn.metrics import r2_score, median_absolute_error, mean_absolute_error, mean_squared_log_error

import numpy as np

In [4]:
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred)/ y_true))*100